In [1]:
## Import packages
import os
import csv
import random
import math
import numpy as np
import pandas as pd
from scipy.io import wavfile
from collections import Counter

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# temp imports
#import resampy
import samplerate
import mel_features
import vggish_params
import wave

In [3]:
def wavfile_to_examples(wav_file):
  """Convenience wrapper around waveform_to_examples() for a common WAV format.
  Args:
    wav_file: String path to a file, or a file-like object. The file
    is assumed to contain WAV audio data with signed 16-bit PCM samples.
  Returns:
    See waveform_to_examples.
  """
  sr, wav_data = wavfile.read(wav_file)
  print("wavfile gelezen")
  assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
  samples = wav_data / 32768.0  # Convert to [-1.0, +1.0]
  log_mel_examples = waveform_to_examples(samples, sr)
  return log_mel_examples

In [4]:
def waveform_to_examples(data, sample_rate):
  """Converts audio waveform into an array of examples for VGGish.
  Args:
    data: np.array of either one dimension (mono) or two dimensions
      (multi-channel, with the outer dimension representing channels).
      Each sample is generally expected to lie in the range [-1.0, +1.0],
      although this is not required.
    sample_rate: Sample rate of data.
  Returns:
    3-D np.array of shape [num_examples, num_frames, num_bands] which represents
    a sequence of examples, each of which contains a patch of log mel
    spectrogram, covering num_frames frames of audio and num_bands mel frequency
    bands, where the frame length is vggish_params.STFT_HOP_LENGTH_SECONDS.
  """
  print("sr: " + str(sample_rate))
  # Convert to mono.
  if len(data.shape) > 1:
    data = np.mean(data, axis=1)
  print("shape = 1")
  # Resample to the rate assumed by VGGish.
  if sample_rate != vggish_params.SAMPLE_RATE:
    #data = resampy.resample(data, sample_rate, vggish_params.SAMPLE_RATE)
    ratio=vggish_params.SAMPLE_RATE/sample_rate #output/input rate
    converter = 'sinc_best' # 'sinc_best' or 'sinc_fastest'
    data = samplerate.resample(data, ratio, converter, False)
    
  print("sample rate klopt")
  # Compute log mel spectrogram features.
  log_mel = mel_features.log_mel_spectrogram(
      data,
      audio_sample_rate=vggish_params.SAMPLE_RATE,
      log_offset=vggish_params.LOG_OFFSET,
      window_length_secs=vggish_params.STFT_WINDOW_LENGTH_SECONDS,
      hop_length_secs=vggish_params.STFT_HOP_LENGTH_SECONDS,
      num_mel_bins=vggish_params.NUM_MEL_BINS,
      lower_edge_hertz=vggish_params.MEL_MIN_HZ,
      upper_edge_hertz=vggish_params.MEL_MAX_HZ)

  # Frame features into examples.
  features_sample_rate = 1.0 / vggish_params.STFT_HOP_LENGTH_SECONDS
  example_window_length = int(round(
      vggish_params.EXAMPLE_WINDOW_SECONDS * features_sample_rate))
  example_hop_length = int(round(
      vggish_params.EXAMPLE_HOP_SECONDS * features_sample_rate))
  log_mel_examples = mel_features.frame(
      log_mel,
      window_length=example_window_length,
      hop_length=example_hop_length)
  return log_mel_examples

In [5]:
## Parameters voor het maken van tf-records
audio_embedding_feature_name = 'audio_embedding'
pca_params = 'models/vggish_pca_params.npz'
checkpoint = 'models/vggish_model.ckpt'
yt_checkpoint = 'models/youtube_model.ckpt'

In [6]:
## Lees alle csv-files in
bal_labels = pd.read_csv('csv_files/balanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

unbal_labels = pd.read_csv('csv_files/unbalanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

eval_labels = pd.read_csv('csv_files/eval_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

In [7]:
## Map waarin yt-wavjes staan
wavfile_path = "wav_files"

In [8]:
## Geef aan of de tf-records voor jungle of urban moeten worden gedownload (ik neem aan dat we alleen jungle doen)
target = "jungle" #"urban"
mid_to_label = pd.read_csv("csv_files/class_labels_indices_" + target + "_prop.csv", sep=";")

In [9]:
## embeddings komen nu in 1 groot tf-record te staan
## We kunnen ook een counter maken: na elke 1000, maak nieuwe tf-record
tfrecord_path = "tfrecords/added_data/"

In [10]:
## Houd aantal samples bij
nr_samples = 0

## Lijst van alle labels (nieuwe) en alle combi's
all_labels = []

## Proportie train/test
pct_train = 0.6

In [11]:
## Voor alle wav-files in een map, gebeurt het volgende:

## Eerst wordt het wav-file ingelezen en de bijbehorende label(s) opgezocht in een csv-bestand
## Als video-id wordt een willekeurig id'tje gegeven (in dit geval allemaal dezelfde)

## Dan parsen we wav-file naar embeddings: 
# (dit gebeurt door vggish_input.wavfile_to_example aan te roepen)
# Stap 1a: lezen van wav-file, input is array met samples die db aanduiden. Ook sample rate (per sec) wordt gelezen
# Stap 1b: Bij 2d array (stereo, ipv mono) bereken gemiddelde, daarna normaliseren (delen door 32.768)
# Stap 2: Bepaal examples in vorm [batch size, num frames, num bands].
    # Hierbij worden voor verschillende batches (omdat alles tegelijk niet in 1x in NN kan),
    # een log mel spectrogram gemaakt (in vorm [num_frames, num_bands])
# Stap 3: Bepaal features: nu wordt de embedding laag gemaakt (PCA-components, discreet maken etc)
    # Hiervoor worden model-parameters opgehaald die eerder zijn opgeslagen
    
## Daarna wordt een sequence example gemaakt (in getSequenceExample) en het als tf-records weggeschreven

In [12]:
## Function that takes examples from wav-file as input and returns a sequence example

def getSequenceExample(examples_batch, labels, video_id=[b'-1LrH01Ei1w']):
    with tf.Graph().as_default(), tf.Session() as sess:

        # Prepare a postprocessor to munge the model embeddings.
        pproc = vggish_postprocess.Postprocessor(pca_params)
    
        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)
        
        # Run inference and postprocessing.
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})
        print(embedding_batch.shape)
        
        postprocessed_batch = pproc.postprocess(embedding_batch)
        #print(postprocessed_batch)


        ## Maak labels en video-id voor in de example
        label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
        videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))
        
        ## Maak sequence example
        seq_example = tf.train.SequenceExample(
            context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
        
    return(seq_example)

In [13]:
## Deze functie checkt of een example wordt gemaakt (downsamplen)
def checkIfNewExample(labels):
    
    ## Alleen bij 1 label worden er examples overgeslagen (of bij write_all_examples)
    if(len(labels)>1):
        return True
    else:
        label = list(labels)[0]
        prop = mid_to_label.loc[mid_to_label['index']==np.int(label), "target_ind"].values[0]
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:
            return True
        else:
            return False
    return False

In [14]:
## Deze functie checkt of er nog een example moet worden gemaakt (upsamplen)
## Momenteel kan het aantal enkel verdubbeld worden

def checkIfExtraExample(labels):
    
    ## Alleen bij 1 label worden er examples gekopieerd (of als er niet (up)gesampled wordt)
    if(len(labels)>1):
        return False
    else:
        label = list(labels)[0]
        prop = mid_to_label.loc[mid_to_label['index']==np.int(label), "target_ind"].values[0]-1
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:    
            return True
        else:
            return False
    return False

In [15]:
def getLabels(mid_str):
    ## Maak lijst van m-id's
    mid_list = mid_str.split(',')
    labels = []
    
    ## Voor elk m-id, vind labels, (if any labels: add to label list)
    for mid in mid_list:
        if (mid_to_label.loc[mid_to_label["mid"] == mid, "mid"].any()):
            labels.append(mid_to_label.loc[mid_to_label["mid"] == mid, "index"].values[0])
    
    ## Return unique set of labels
    return set(labels)

In [16]:
def readWav(vid_to_mid, dataset):
    
    train_tfrecord = str(tfrecord_path + dataset + '_train.tfrecord')
    test_tfrecord = str(tfrecord_path + dataset + '_test.tfrecord')
    
    train_writer = tf.python_io.TFRecordWriter(train_tfrecord)
    test_writer = tf.python_io.TFRecordWriter(test_tfrecord)
    
    ## Read and rewrite all test files
    files = gfile.Glob(str(wavfile_path + "/" + dataset + "/*.wav"))
  
    count=0
    for file in files:
    
        count+=1
        print("\nfile nr: " + str(count))
        filename = file.split("\\")[-1]
        print("Filename: " + filename)
        filename = filename[0:-4]
        ytid = filename.split("_^_")[0]
        print("ytid: " + str(ytid))
        start = filename.split("_^_")[1]
        print("start: " + str(start))
        
        ## Vind bijbehorende m-ids
        mids = vid_to_mid.loc[vid_to_mid["YTID"] == ytid, "positive_labels"].values[0]
        print('test - mids: ' + str(mids))
        new_labels = getLabels(mids)
        print('test - labels:' + str(new_labels))
        
        # check duration>0
        f = wave.open(file)
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
        print('test - duration: ' + str(duration))
        f.close()
        
        if(duration>0):
            ## This function reads the wav file and converts the samples into np arrays of [batch size, num frames, num bands]
            #examples_batch = vggish_input.wavfile_to_examples(str(wavfile_path + wav_file))
            #examples_batch = vggish_input.wavfile_to_examples(file)
            examples_batch = wavfile_to_examples(file)

            ## Bij meer dan 10 sec: neem middelste 10 sec
            print("test - check: seconds")
            if(examples_batch.shape[0]>10):
                nr_sec = examples_batch.shape[0]
                start = int(math.floor((nr_sec-10)/2))
                end = int(nr_sec-math.ceil((nr_sec-10)/2))
                examples_batch = examples_batch[start:end, :, :]

            ## Schrijf alleen bij meer dan 5 sec
            ## Check if new example: voor downsamplen
            print("check: write example")
            if(examples_batch.shape[0]>5 and checkIfNewExample(new_labels)):

                all_labels.extend(new_labels)

                seq_example = getSequenceExample(examples_batch, new_labels)

                rand = random.random()
                if rand <= pct_train:
                    train_writer.write(seq_example.SerializeToString())
                else:
                    test_writer.write(seq_example.SerializeToString())

                ## Voor upsamplen
                print("check: upsamplen")
                if(checkIfExtraExample(new_labels)):
                    rand = random.random()
                    if rand <= pct_train:
                        train_writer.write(seq_example.SerializeToString())
                    else:
                        test_writer.write(seq_example.SerializeToString())

                    all_labels.extend(new_labels)
                
    train_writer.close()
    test_writer.close()

In [17]:
readWav(bal_labels, "bal")
#readWav(unbal_labels, "unbal")
#readWav(eval_labels, "eval")


file nr: 1
Filename: --aaILOrkII_^_200.000.wav
ytid: --aaILOrkII
start: 200.000
test - mids: /m/032s66,/m/073cg4
test - labels:{3}
test - duration: 10.0
wavfile gelezen
sr: 44100
shape = 1
sample rate klopt
test - check: seconds
check: write example

file nr: 2
Filename: --aO5cdqSAg_^_30.000.wav
ytid: --aO5cdqSAg
start: 30.000
test - mids: /t/dd00003,/t/dd00005
test - labels:{5}
test - duration: 10.0
wavfile gelezen
sr: 44100
shape = 1
sample rate klopt
test - check: seconds
check: write example

file nr: 3
Filename: -24dqQM_rDk_^_30.000.wav
ytid: -24dqQM_rDk
start: 30.000
test - mids: /m/04rlf,/t/dd00003
test - labels:{5}
test - duration: 10.0
wavfile gelezen
sr: 44100
shape = 1
sample rate klopt
test - check: seconds
check: write example

file nr: 4
Filename: -3IGxVTJvgI_^_30.000.wav
ytid: -3IGxVTJvgI
start: 30.000
test - mids: /m/06bz3,/m/09x0r
test - labels:{7}
test - duration: 0.0

file nr: 5
Filename: -60XojQWWoc_^_30.000.wav
ytid: -60XojQWWoc
start: 30.000
test - mids: /m/02rhd

In [18]:
print("nr samples: " + str(nr_samples))

nr samples: 0


In [19]:
## Print voor alle labels het aantal voorkomens in de nieuwe tf-records

sum_occur = 0
cnt_labels = Counter(all_labels)

for i in cnt_labels.most_common(100):
    sum_occur = sum_occur + i[1]
    print(label_proportions.loc[label_proportions['index']==i[0]]['display_name'].values[0], ' : ', str(i[1]))

NameError: name 'Counter' is not defined